In [5]:
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime
from unidecode import unidecode

import pycep_correios as pycep

import requests as rq
import bs4
import json
import tqdm
import glob
import os

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

# Lendo o arquivo

In [6]:
path = './data/01_raw_df/'

# Nome dos arquivos dentro do diretorio acima (raw_df)
filenames = os.listdir(path)

# Dataframe dos dados sem tratamento 
df = pd.read_csv(path + filenames[-1], sep='|') # -1 para pegar o ultimo dataframe adquirido
# Dataframe dos dados ja adquiridos
cep_df= pd.read_json(f'./03_parsed_ceps/parsed_cep_details.json', lines=True).dropna(how= 'all')
cep_df=cep_df.rename(columns = {'cep': 'zipcode'})

sem_cep = df['zipcode'].isnull().sum()

## Itens sem CEP

In [7]:
print('Quantidade de itens sem CEP:', sem_cep)

Quantidade de itens sem CEP: 85


# Adquirindo dados do endereço via API www.cepaberto.com

In [8]:
# Selecionando apenas os itens com CEP
df_temp = df[df['zipcode'].notna()]

# Usando a API: https://www.cepaberto.com/
# Ref https://github.com/edumucelli/cepaberto-api-examples/blob/master/api/v3/api.py
token = "262cd9ff6f7507bf4030953c5da76e79"
headers = {'Authorization': 'Token token=%s' % token}

def geoloc_by_cep(cep):
    cep = str(int(cep)).zfill(8) # Preenchendo zeros a esquerda para o formato CEP
    url = f'https://www.cepaberto.com/api/v3/cep?cep={cep}'
    response = rq.get(url, headers=headers)
    time.sleep(1)
    return response.json()

In [11]:
geoloc_by_cep(1323020) # Exemplo de output

{'altitude': 760.0,
 'cep': '01323020',
 'latitude': '-23.5350399951',
 'longitude': '-46.6501372767',
 'logradouro': 'Rua João Julião',
 'bairro': 'Bela Vista',
 'cidade': {'ddd': 11, 'ibge': '3550308', 'nome': 'São Paulo'},
 'estado': {'sigla': 'SP'}}

In [10]:
# Ceps já adquiridos anteriormente
ceps_antigos= pd.read_json(f'./03_parsed_ceps/parsed_cep_details.json', lines=True)['cep']
# Ceps adquiridos no scrapping
ceps_novos = df_temp['zipcode'].unique()
# Ceps ainda nao adquiridos
ceps = list(set(ceps_novos).difference(set(ceps_antigos)))

print(f'quantidade de ceps para adquirir: {len(ceps)}')

for i in tqdm.tqdm_notebook(ceps):
    try:
        data = geoloc_by_cep(i)
    except:
        print('\n===== Erro! =====\n')
        
    # Checando se o dicionario veio vazio (nao retornou nada)
    if bool(data) == True: # True --> nao retornou vazio
        print(f'CEP: {int(i)}')
        with open(f"./03_parsed_ceps/parsed_cep_details.json", "a+") as output:
            output.write("{}\n".format(json.dumps(data)))
            print(data, '\n')
    else:
        print(f'CEP {int(i)} não encontrado \n')

quantidade de ceps para adquirir: 44


  0%|          | 0/44 [00:00<?, ?it/s]

CEP 1307010 não encontrado 

CEP: 1323020
{'altitude': 760.0, 'cep': '01323020', 'latitude': '-23.5350399951', 'longitude': '-46.6501372767', 'logradouro': 'Rua João Julião', 'bairro': 'Bela Vista', 'cidade': {'ddd': 11, 'ibge': '3550308', 'nome': 'São Paulo'}, 'estado': {'sigla': 'SP'}} 

CEP: 1046925
{'altitude': 760.0, 'cep': '01046925', 'latitude': '-23.548', 'longitude': '-46.6388470922', 'logradouro': 'Avenida Ipiranga 200', 'bairro': 'República', 'cidade': {'ddd': 11, 'ibge': '3550308', 'nome': 'São Paulo'}, 'estado': {'sigla': 'SP'}} 

CEP: 1330060
{'altitude': 760.0, 'cep': '01330060', 'latitude': '-23.5343199959', 'longitude': '-46.6210773202', 'logradouro': 'Rua Una', 'bairro': 'Bela Vista', 'cidade': {'ddd': 11, 'ibge': '3550308', 'nome': 'São Paulo'}, 'estado': {'sigla': 'SP'}} 

CEP: 1136030
{'altitude': 760.0, 'cep': '01136030', 'latitude': '-23.548', 'longitude': '-46.6413014861', 'logradouro': 'Rua Gaspar Ricardo Júnior', 'bairro': 'Barra Funda', 'cidade': {'ddd': 11, 

In [12]:
# Dataframe dos dados ja adquiridos
cep_df= pd.read_json(f'./03_parsed_ceps/parsed_cep_details.json', lines=True).dropna(how= 'all')
cep_df=cep_df.rename(columns = {'cep': 'zipcode'})
cep_df['zipcode'] = cep_df['zipcode'].astype(int)

df_joined = pd.merge(df, cep_df, on='zipcode')

In [13]:
now = datetime.now().strftime("%m-%d-%Y_%H%M%S") # Data e horario
df_joined.to_csv(f'./data/02_raw_df_w_end/raw_df_w_end_{now}.csv', sep='|',  index=False) # Cria o dataframe